# Netflix movie recommendation engine
Kaggle competition link: https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data 

After doing some preprocessing, now we have 2 datasets in sparse matrix format. 
1. train_sparse_matrix
2. test_sparse_matrix

Both the data sets have 'user' as rows and 'movie' as columns and 'rating' as data

In [4]:
# imports
from datetime import datetime # To compute time taken wherever necessary
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import sparse
from scipy.sparse import csr_matrix

import warnings
warnings.filterwarnings("ignore")

In [5]:
# Load train & test sparse matrix which we've created in previous file
transformed_folder = 'F:/09 - Machine Learning Case Studies/01 - Netflix Movies Recommendation/Transformed Data'
train_sparse_matrix = sparse.load_npz(transformed_folder+'/train_sparse_matrix.npz')
print('train_sparse_matrix loaded!')
test_sparse_matrix = sparse.load_npz(transformed_folder+'/test_sparse_matrix.npz')
print('test_sparse_matrix loaded!')

train_sparse_matrix loaded!
test_sparse_matrix loaded!


### 2. Feature Engineering

#### 2.1 Basic features based on statistics
Lets try to build some features that will be useful in modelling. Few such features could be:
1. Average of all ratings given
2. Average rating per user
3. Average rating per movie

We will create a dictionary train_averages which will store all these values. Like:<br>
train_averages = <br>
{<br>
'global_average' : xyz,<br>
'user' : { 'user1': abc, 'user2':pqr},<br>
'movie' : { 'movie1': abc, 'movie2':pqr}<br>
}

In [6]:
train_averages = dict()

#### Global average

In [7]:
start = datetime.now()
# Global average
train_averages['global_average'] = train_sparse_matrix.sum()/train_sparse_matrix.count_nonzero()
print(train_averages)
print("*"*50)
print('Time taken: {}'.format(datetime.now() - start))

{'global_average': 3.582890686321557}


#### Average rating per user

In [36]:
start = datetime.now()
# Average rating per user
print('Computing average ratings per user...')
sum_of_ratings_per_user = train_sparse_matrix.sum(axis=1).A1
no_of_ratings_per_user = (train_sparse_matrix!=0).sum(axis=1).A1

u,m = train_sparse_matrix.shape
average_ratings = dict()
for i in range(0,u):
    if no_of_ratings_per_user[i]!=0:
        average_ratings[i] = sum_of_ratings_per_user[i]/no_of_ratings_per_user[i]

train_averages['user'] = average_ratings
print('Done!')
print("*"*50)
print('Time taken: {}'.format(datetime.now() - start))

Computing average ratings per user...
Done!
**************************************************
Time taken: 0:00:01.172182


In [49]:
print('Average rating given by user 25 is: {}'.format(train_averages['user'][25]))

Average rating given by user 25 is: 3.5


#### Average rating per movie

In [38]:
start = datetime.now()
# Average rating per user
print('Computing average ratings per movie...')
sum_of_ratings_per_movie = train_sparse_matrix.sum(axis=0).A1
no_of_ratings_per_movie = (train_sparse_matrix!=0).sum(axis=0).A1

u,m = train_sparse_matrix.shape
average_ratings = dict()
for i in range(0,m):
    if no_of_ratings_per_movie[i]!=0:
        average_ratings[i] = sum_of_ratings_per_movie[i]/no_of_ratings_per_movie[i]

train_averages['movie'] = average_ratings
print('Done!')
print("*"*50)
print('Time taken: {}'.format(datetime.now() - start))

Computing average ratings per movie...
Done!
**************************************************
Time taken: 0:00:00.556723


In [50]:
print('Average rating given for movie 50 is: {}'.format(train_averages['movie'][50]))

Average rating given for movie 50 is: 2.9930795847750864


#### 2.2 Compute user-user similarity matrix

We know that there are more than 400k users. If we try to compute similarity using cosine similarity, we will need to do (400k * 400k / 2) = 80 billions computations, which will take days to compute even though it will only perform on nonzeros. <br>
If we try to reduce number of dimensions using PCA or SVD, it will take even more time as the matrix will become dense and the multiplication will be done for each feature as there will not be zero cells. <br><br>

One of the ideas is to compute similarity at <b>run time</b>. Here, we will follow below startegy to compute user-user similarity:<br>
1. We will compute similarity (top N) for given user (run time)
2. Once we compute similarity for any user, we will store this data in our customized data structure so that we can retirve it whenever we want in future
3. Now next time if we want to compute similarity for any user, first we will check if we have already computed for that particular user. If yes, then get it from out customized data structure. If not, compute the similarity and store it in our customized data structure.
<br><br>
Our customized data structure will be a <b>dictionary of dictionaries</b><br>
{ 'user1': {'similar_user1' : value1 }, {'similar_user2' : value2 }, {'similar_user3' : value3 } <br>
  'user2': {'similar_user1' : value1 }, {'similar_user2' : value2 }, {'similar_user3' : value3 } }

#### 2.3 Compute movie-movie similarity matrix

We know that there are around 17k movies in total. If we try to compute cosine similarity, we will need to do (17k * 17k / 2) = 144 millions computations, which would not take much longer. In this case, we can definately go with cosine similarity to get similar movies for each movie.